In [1]:
import os
import cv2

# Pfade anpassen
image_folder = r"C:\Uni\python\data\Klassifikation\33_VorgeschriebeneFahrtrichtungRechts\33"
label_folder = r"C:\Uni\python\data\Klassifikation\33_labels\yolo_labels\renamed_labels"
output_folder = r"C:\Uni\python\data\Klassifikation\33_cropped_images"

# Output-Ordner erstellen, falls nicht vorhanden
os.makedirs(output_folder, exist_ok=True)

# Bild- und Label-Verarbeitung
for label_file in os.listdir(label_folder):
    if label_file.endswith(".txt"):  # Nur Label-Dateien verarbeiten
        image_name = os.path.splitext(label_file)[0] + ".png"  # Erwarte Bildformat .png
        image_path = os.path.join(image_folder, image_name)
        label_path = os.path.join(label_folder, label_file)

        # Bild laden
        if os.path.exists(image_path):
            image = cv2.imread(image_path)
            h, w, _ = image.shape  # Höhe und Breite des Bildes

            # Labels laden und zuschneiden
            with open(label_path, "r") as file:
                lines = file.readlines()
                for line in lines:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())

                    # YOLO-Koordinaten in Pixel umrechnen
                    x_center, y_center = int(x_center * w), int(y_center * h)
                    width, height = int(width * w), int(height * h)
                    x1, y1 = max(0, x_center - width // 2), max(0, y_center - height // 2)
                    x2, y2 = min(w, x_center + width // 2), min(h, y_center + height // 2)

                    # Zuschneiden und speichern
                    cropped_image = image[y1:y2, x1:x2]
                    output_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_cropped.png")
                    cv2.imwrite(output_path, cropped_image)
        else:
            print(f"Bild {image_name} nicht gefunden!")


In [2]:
import os
import shutil

# Pfade anpassen
image_folder = r"C:\Uni\python\data\Klassifikation\33_cropped_images"
label_folder = r"C:\Uni\python\data\Klassifikation\33_labels\yolo_labels\renamed_labels"
output_folder = r"C:\Uni\python\data\Klassifikation\classification_ready"

# Output-Ordner erstellen
os.makedirs(output_folder, exist_ok=True)

# Bilder nach Klassen sortieren
for label_file in os.listdir(label_folder):
    if label_file.endswith(".txt"):
        image_name = os.path.splitext(label_file)[0] + "_cropped.png"
        image_path = os.path.join(image_folder, image_name)
        label_path = os.path.join(label_folder, label_file)

        if os.path.exists(image_path):
            # Klassen-ID aus dem Label lesen
            with open(label_path, "r") as file:
                first_line = file.readline()
                class_id = first_line.split()[0]  # Erste Zahl ist die Klassen-ID

            # Klasse als Ordner erstellen
            class_folder = os.path.join(output_folder, class_id)
            os.makedirs(class_folder, exist_ok=True)

            # Bild in den Klassenordner verschieben
            shutil.move(image_path, os.path.join(class_folder, image_name))
        else:
            print(f"Bild {image_name} nicht gefunden!")
